<font size=8>  Setting up the Collection Space Navigator

In this How-To guide you will produce all necessary files to create a custom version of the Collection Space Navigator (CSN).

Project link: https://collection-space-navigator.github.io/ 

> Note: We highly recommend to first get familiar with the example collection before trying your with your own data.

In [ ]:
from IPython.display import HTML
    
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="toggle on/off raw code"></form>''')

<font size=6> 1) Prepare Collection Data

Loads the dataset and sets up the metadata.



### Import Libraries
>Note: Optional libraries will be installed only if needed.

In [1]:
!git clone https://github.com/Collection-Space-Navigator/CSN
%cd CSN

import json, math, os, io
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import interactive,HBox,VBox,Label
from IPython.display import display

### Define INPUT
>Note: Running this cell opens a dialog in which the input data can be defined. Use the provided example data (recommended first) or your own.


In [2]:
style = {'description_width': '250px'}
layout = {'width': '600px', 'justify-content': 'lex-satrt'}
# layoutButtons = {'width': '210px'}
useExample = widgets.Checkbox(value=True,description='use example data',indent=True)
datasetTitle = widgets.Text(placeholder='title of the dataset', description='Title:', style=style, layout=layout, value = "Testset")
description = widgets.Textarea(placeholder='Short description of the dataset and method(s)', description='Description (optional):', style=style, layout=layout, value="")
embeddingsLocation = widgets.Text(placeholder='path to embeddings file (.csv)', description='Embeddings Filepath (optional):', style=style, layout=layout, value = "CSN/example_data/embeddings_testset.csv")
metadataLocation = widgets.Text(placeholder='path to metadata file (.csv)', description='Metadata Filepath:', style=style, layout=layout, value = "CSN/example_data/metadata_testset.csv")
imageLocation = widgets.Text(placeholder='path to image collection folder', description='Image Folder:', style=style, layout=layout, value = "CSN/example_data/testset_images/")
imageWebLocation = widgets.Text(placeholder='URL prefix to public image directory', description='Image URL prefix:',style=style, layout=layout, value =  "https://github.com/Collection-Space-Navigator/CSN/raw/main/example_data/testset_images/")

if os.path.exists("/content/gdrive/MyDrive/"):
    imageLocation.value = "/content/gdrive/MyDrive/CSN/example_data/testset_images/"
    embeddingsLocation.value = "/content/gdrive/MyDrive/CSN/example_data/embeddings_testset.csv"
    metadataLocation.value = "/content/gdrive/MyDrive/CSN/example_data/metadata_testset.csv"  
else:
    imageLocation.value = "example_data/testset_images/"
    embeddingsLocation.value = "example_data/embeddings_testset.csv"
    metadataLocation.value = "example_data/metadata_testset.csv"  

imageWebLocation.value =  "https://github.com/Collection-Space-Navigator/CSN/raw/main/example_data/testset_images/"


subset = widgets.Checkbox(value=False,description='make subset',indent=True)
subsetSize = widgets.BoundedIntText(value=2048,min=10,max=9999999, step=1,description='Subset size:')
def makeSubset(SUBSET):
    if SUBSET:
        display(subsetSize)
    else:
        subsetSize.value == None
i = interactive(makeSubset, SUBSET = subset)
left = VBox([datasetTitle, description, embeddingsLocation, metadataLocation, imageLocation, imageWebLocation])
right = VBox([useExample, i])
display(HBox([left,right]))

### Load INPUT files
Loads and checks all files.

>Note: Example data will be downloaded only if needed.

In [3]:

imagNumb = len(os.listdir(imageLocation.value))
print(f'found {imagNumb} files in {imageLocation.value}')

metadata = pd.read_csv(metadataLocation.value, skipinitialspace=True)
if subset:
    metadata = metadata[:subsetSize.value]
metaNumb = len(metadata)
print(f'found {metaNumb} entries in {metadataLocation.value}')

if embeddingsLocation.value != "":
  embeddings = pd.read_csv(embeddingsLocation.value, skipinitialspace=True)
  embeddings = embeddings.loc[:, embeddings.columns!='id']
  if subset:
    embeddings = embeddings[:subsetSize.value]
  vecNumb = len(embeddings)
  print(f'found {vecNumb} entries in {embeddingsLocation.value}')

  if metaNumb == vecNumb:
    if vecNumb <= imagNumb:
      print("Looks ok.")
      print()
      print(f'Embedding file contains {vecNumb} vectors in {len(embeddings.columns)} dimensions.')
      print("Metadata Head:")
      print(metadata.head())
    else:
      print()
      print("ERROR: number of images is smaller than number of vectors")

if metaNumb <= imagNumb:
  print("Looks ok.")
  print("Metadata Head:")
  print(metadata.head())
else:
  print()
  print("ERROR: number of images and metadata elements don't match!")
foldername = datasetTitle.value.lower().replace(" ","_")
print()
print(f'Creating new dataset directory: build/datasets/{foldername}...')
if not os.path.exists(f"build/datasets/{foldername}"):
    os.makedirs(f"build/datasets/{foldername}")
    print("... success")
else:
    print("... folder already exists (might overwrite existing files)")

created new directory 'CSN'
/Users/tillmannohm/Documents/CollectionSpaceNavigator/WikiArt_demo/CSN
Initialized empty Git repository in /Users/tillmannohm/Documents/CollectionSpaceNavigator/WikiArt_demo/CSN/.git/
Updating origin
remote: Enumerating objects: 2258, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 2258 (delta 27), reused 37 (delta 12), pack-reused 2194
Receiving objects: 100% (2258/2258), 590.78 MiB | 2.17 MiB/s, done.
Resolving deltas: 100% (675/675), done.
From https://github.com/Collection-Space-Navigator/CSN
 * [new branch]      gh-pages   -> origin/gh-pages
 * [new branch]      main       -> origin/main
downloading example dataset...
From https://github.com/Collection-Space-Navigator/CSN
 * branch            main       -> FETCH_HEAD
...done
/Users/tillmannohm/Documents/CollectionSpaceNavigator/WikiArt_demo
found 1000 files in CSN/example_data/testset_images/
found 1000 entries in CSN/example_data/metada

### Assign metadata fields
Choose which field names in the metadata file should be used.   

>Note: Select multiple values using ctrl+click, command+mouseclick, or shift+arrow keys.


In [4]:

filenameColumn = widgets.Dropdown(description="Image filenames (JPG or PNG):",options=[mf for mf in metadata.columns if pd.api.types.is_string_dtype(metadata[mf]) and metadata[mf].str.endswith((".jpg",".JPEG","JPG",".jpeg",".png",".PNG")).all()], style=style, layout=layout)
classColumns = widgets.SelectMultiple(options=[mf for mf in metadata.columns if pd.api.types.is_integer_dtype(metadata[mf]) and mf != "index"],description='optional: Cluster data (integers):', style=style, layout=layout)
infoColumns = widgets.SelectMultiple(options=[mf for mf in metadata.columns if mf != "index"],description='Info fields (display in preview):', style=style, layout=layout)
sliderColumns = widgets.SelectMultiple(options=[mf for mf in metadata.columns if pd.api.types.is_numeric_dtype(metadata[mf]) and mf != "index"],description='Slider data (floats or integers):', style=style, layout=layout)
filterColumns = widgets.SelectMultiple(options=[mf for mf in metadata.columns if pd.api.types.is_string_dtype(metadata[mf]) and mf != 'URL'],description='optional: Filter & Search fields (string):', style=style, layout=layout)
if useExample.value == True:
  infoColumns.value = ("Prompt", "Colors", "Contrast", "File Size")
  sliderColumns.value = ("Colorfulness", "Colors", "Contrast", "File Size")
  filterColumns.value = ("Prompt",)
  classColumns.value = ("Class",)
left = VBox([filenameColumn, infoColumns, sliderColumns])
right = VBox([filterColumns, classColumns])
display(HBox([left,right]))


----------------

<font size=6> 2) Prepare Image Data

To handle large amounts of images efficiently, the CSN uses sprite sheets with multiple thumbnails behind the scenes. These sprite-sheets need to be generated.

### Generate sprite sheets
>Note: only needed for new datasets or to update existing tiles (skip this part if you already generated them)


In [5]:

from CSN_utils import ImageTileGenerator

tile_generator = ImageTileGenerator(foldername, tileSize=2048, tileRows=32, imageFolder=imageLocation.value, files=metadata[filenameColumn.value]).generate()

Generating tiles: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.96s/it]


----------------

<font size=6> 3) Generate Mappings

Mappings are plots containing 2D coordinates (x,y) of the image objects. 

Here are several methods you can run. The Collection Space Navigator can handle many mappings but needs at least one to work.

<font size=5> 3.1 From metadata (optional)

### (optional) Create 2D plots
Choose 2 metadadata fields (float or integer) and click "make plot". Repeat for every combination you want to add.
>Note: Running this step opens a dialog in which you can chose X and Y dimensions from the available data and create additional 2D plots


In [6]:

from CSN_utils import SimplePlot

def plot(v):
    SimplePlot(foldername,A=AColumn.value, B=BColumn.value, metadata=metadata)
    
AColumn = widgets.Dropdown(description="x-axis:",options=[mf for mf in metadata.columns if pd.api.types.is_numeric_dtype(metadata[mf]) and mf != "index"], style=style, layout=layout)
BColumn = widgets.Dropdown(description="y-axis:",options=[mf for mf in metadata.columns if pd.api.types.is_numeric_dtype(metadata[mf]) and mf != "index"], style=style, layout=layout)
button2DPlot = widgets.Button(description='make plot',icon='check')
button2DPlot.on_click(plot)
left = VBox([AColumn,BColumn])
right = VBox([button2DPlot])
HBox([left,right])

normalized plot
centered embedding
saved Colorfulness_Colors.json


<font size=5> 3.2 From embeddings (optional)

### (optional) Run Principal Component Analysis (PCA)
>See PCA documentation: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.eA.html


In [7]:
components = 3
add_slider = False

from CSN_utils import PCAGenerator

PCAEembedding = PCAGenerator(foldername, scale=True, data=embeddings.values, components=components).generate()

# add columns to metadata for each component
for i in range(components):
  metadata[f"PC{i+1}"] = PCAEembedding[:,i]
  print(f"... added PC{i+1} to metadata")

# add slider for each component
if add_slider:
  sliderCols = list(sliderColumns.value)
  for i in range(components):
    sliderCols.append(f"PC{i+1}")

Performing PCA...
...done
saved PCA.json
... added PC1 to metadata
... added PC2 to metadata
... added PC3 to metadata


### (optional) Run UMAP: Uniform Manifold Approximation and Projection for Dimension Reduction
>See UMAP documentation: https://umap-learn.readthedocs.io/en/latest/


In [8]:
n_neighbors=15 
min_dist=0.18 
metric="correlation"
verbose=True


from CSN_utils import UMAPGenerator

fullEmbeddings = UMAPGenerator(foldername, data=embeddings.values, n_neighbors=n_neighbors, min_dist=min_dist, metric=metric, verbose=verbose).generate()


Generating UMAP...
UMAP(angular_rp_forest=True, metric='correlation', min_dist=0.18, verbose=True)
Wed Mar 29 16:43:04 2023 Construct fuzzy simplicial set
Wed Mar 29 16:43:07 2023 Finding Nearest Neighbors
Wed Mar 29 16:43:09 2023 Finished Nearest Neighbor Search
Wed Mar 29 16:43:11 2023 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

Wed Mar 29 16:43:14 2023 Finished embedding
...done
saved UMAP.json


### (optional) Run t-SNE: t-distributed Stochastic Neighbor Embedding
>See t-SNE documentation: https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html

In [9]:
n_components = 2 
verbose = 1
random_state = 123

from CSN_utils import TSNEGenerator

tsneEembedding = TSNEGenerator(foldername, data=embeddings.values, n_components=n_components, verbose=verbose, random_state=random_state).generate()

Generating t-SNE...
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.001s...
[t-SNE] Computed neighbors for 1000 samples in 0.058s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 14.950695


/Users/tillmannohm/Documents/coding/ARCU/env/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/Users/tillmannohm/Documents/coding/ARCU/env/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


[t-SNE] KL divergence after 250 iterations with early exaggeration: 83.809944
[t-SNE] KL divergence after 1000 iterations: 1.184980
...done
saved TSNE.json


### (optional) Create 2D plots
Choose 2 metadadata fields (float or integer) and click "make plot". Repeat for every combination you want to add.
>Note: Running this step opens a dialog in which you can chose X and Y dimensions from the available data and create additional 2D plots


In [ ]:

from CSN_utils import SimplePlot

def plot(v):
    SimplePlot(foldername,A=AColumn.value, B=BColumn.value, metadata=metadata)
    
AColumn = widgets.Dropdown(description="x-axis:",options=[mf for mf in metadata.columns if pd.api.types.is_numeric_dtype(metadata[mf]) and mf != "index"], style=style, layout=layout)
BColumn = widgets.Dropdown(description="y-axis:",options=[mf for mf in metadata.columns if pd.api.types.is_numeric_dtype(metadata[mf]) and mf != "index"], style=style, layout=layout)
button2DPlot = widgets.Button(description='make plot',icon='check')
button2DPlot.on_click(plot)
left = VBox([AColumn,BColumn])
right = VBox([button2DPlot])
HBox([left,right])

----------------

<font size=6> 4) Create Config Files

All customization and component settings are defined in the config files.

### Set Sliders
Set the appearance of the range slider elements and histograms.

In [10]:

try:
  import distinctipy
except:
  print("Installing distinctipy via Pip")
  !pip install distinctipy --quiet
  import distinctipy
  
# check if sliderCols exists
try:
  sliderCols
except NameError:
  sliderCols = list(sliderColumns.value)
  
if len(sliderCols) > 0:    
  layoutCol = {'width': '110px'}
  sliderColorDict = {}
  left = [Label('display name')]
  middle = [Label('description text')]
  right = [Label('histogram color')]
  colors = distinctipy.get_colors(len(sliderCols),pastel_factor=1)
  for i, sliderName in enumerate(sliderCols):
    sliderColorDict[sliderName] = widgets.ColorPicker(concise=False,value=distinctipy.get_hex(colors[i]),layout=layoutCol)
    right.append(sliderColorDict[sliderName])
  sliderInfoDict = {}
  for sliderName in sliderCols:
    sliderInfoDict[sliderName] = widgets.Text(placeholder="info text for slider",layout=layout)
    middle.append(sliderInfoDict[sliderName])
  sliderNameDict = {}
  for sliderName in sliderCols:
    sliderNameDict[sliderName] = widgets.Text(placeholder="name of slider",value=sliderName)
    left.append(sliderNameDict[sliderName])
  print("\nSlider Settings:\n") 
  idx = VBox([Label('')]+[Label(f"{n}:") for n in sliderCols])
  left_box = VBox([l for l in left])
  middle_box = VBox([m for m in middle])
  right_box = VBox([r for r in right])
  display(HBox([idx,left_box,middle_box,right_box]))
else:
  print("No Cluster fields selected!")


Slider Settings:



### (optional) Set Cluster colors
>Note: only necessary if categorical data was assigned for clusters

In [11]:

if len(classColumns.value) > 0:
  classColorDict = {}
  amount = len(classColumns.value)
  styleCol = {'description_width': '25px'}
  layoutCl = {'width': '135px'}
  allClasses = {}
  for className in classColumns.value:
    clusters = metadata[className].unique()
    allClasses[className] = len(clusters)
  l = sorted(allClasses.items(), key=lambda item: item[1])[0]
  length = l[1]
  allColors = {}
  colors = distinctipy.get_colors(length)
  col = 5
  row = math.ceil(length/col)
  i=0
  rows = []
  for r in range(0,col):
    newRow = []
    for c in range(0,row):
      # classColorDict[className] = widgets.ColorPicker(concise=True, value=distinctipy.get_hex(colors[i]))
      if i < len(colors):
        allColors[i] = widgets.ColorPicker(concise=False, description=str(i), value=distinctipy.get_hex(colors[i]),layout=layoutCl,style=styleCol)
        newRow.append(allColors[i])
        i+=1
    rows.append(VBox([nr for nr in newRow]))
  display(HBox(rows))
else:
  allColors = False
  print("No cluster was selected.")


### Create metadata.json
Creates and saves the metadata.json file
>Note: This step is necessary!



In [12]:

from CSN_utils import Utils

try:
  sliderCols
except NameError:
  sliderCols = list(sliderColumns.value)

imageFolder = f'public/datasets/{foldername}/images/'
if useExample.value == True:
  metadata["URL"] = metadata[filenameColumn.value]
else:
  metadata["URL"] = f"{imageFolder}/{metadata[filenameColumn.value]}"
metadataColumns = set(list(infoColumns.value) + sliderCols + list(filterColumns.value) + list(classColumns.value))
metadataColumns.add(filenameColumn.value)
metadata = metadata[metadataColumns]
Utils.write_metadata(foldername, metadata)

saved metadata.json


### Calculate Histograms and create config files
The CSN features Range Sliders with interactive histograms. This step calculates the necessary bins and prepares the data to display the histograms.

In [13]:

try:
    sliderCols
except NameError:
    sliderCols = list(sliderColumns.value)


from CSN_utils import HistogramGenerator, Utils

BarChartData = HistogramGenerator(foldername, data=metadata, selection=sliderCols, bucketCount = 50).generate()

def save_datasetsJSON():
  with open(f'build/datasets/datasets_config.json', "w") as fd:
    json.dump(datasetsJSON , fd)
  print("saved datasets_config.json")

def make_default(DEFAULT):
  datasetsJSON["default"] = DEFAULT
  print(f"changed default dataset to {datasetsJSON['data'][DEFAULT]['name']}")
  save_datasetsJSON()
  

sliderSetting = []

for k in sliderCols:
  dtype = 'float'
  if pd.api.types.is_integer_dtype(metadata[k]):
    dtype = 'int'
  slider = {"id":k,"title":sliderNameDict[k].value,"info":sliderInfoDict[k].value,"typeNumber":dtype,"color":sliderColorDict[k].value}
  sliderSetting.append(slider)
searchFields = []
for k in filterColumns.value:
  filter = {"columnField":k,"type":"selection"}
  searchFields.append(filter)
try:
    allColors
except NameError:
    clusters = {"clusterList":[],"clusterColors":[]}
else:
    clusters = {"clusterList":list(classColumns.value),"clusterColors":[allColors[g].value for g in allColors]}

configData = Utils.write_config(foldername, title=datasetTitle.value, description=description.value, clusters=clusters, total=len(metadata), sliderSetting=sliderSetting, infoColumns=infoColumns.value, searchFields=searchFields, imageWebLocation=imageWebLocation.value)
newDataset = {'name': datasetTitle.value, 'directory': foldername}

datasetsJSON = {"default": 0, "data": [newDataset]}
save_datasetsJSON()

preparing Slider Bar Historgram data Colorfulness
preparing Slider Bar Historgram data Colors
preparing Slider Bar Historgram data Contrast
preparing Slider Bar Historgram data File Size
saved barData.json
saved config.json
saved datasets_config.json

Continue with building the Collection Space Navigator in the next step...


----------------

<font size=6> 5) Test and use your custom Collection Space Navigator

## (optional) Run in localhost

>Note: only works locally (not within Colab)

To run your CSN version on localhost, unzip your downloaded file, open a terminal, navigate to your CSN directory and run `serve -s build`

The CSN should be then accessible at http://localhost:3000 in your browser.



In [16]:
from flask import Flask, render_template,send_from_directory

app = Flask(__name__, static_folder='CSN/build/static',template_folder='CSN/build/')

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/<path:path>')
def send_report(path):
  # remove the replace in next to lines later later <-- important !!!!!!!!
  print("files_report:",path)
  if path=="CSN/manifest.json":
      path="manifest.json"
  return send_from_directory('CSN/build/', str(path))

@app.route('/CSN/static/<path:path>')
def send_report2(path):
  # remove the replace in next to lines later later <-- important !!!!!!!!
  print("files_report2:",path)
  return send_from_directory('CSN/build/static/', str(path))

@app.route('/CSN/datasets/<path:path>')
def send_report3(path):
  # remove the replace in next to lines later later <-- important !!!!!!!!
  print("files_report3:",path)
  return send_from_directory('CSN/build/datasets/', str(path))

if __name__ == "__main__":
  #app.run(debug=False, port=port)
  app.run(debug=False, port=8000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8001
Press CTRL+C to quit
127.0.0.1 - - [29/Mar/2023 16:44:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2023 16:44:33] "GET /static/css/main.9fa499eb.chunk.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2023 16:44:33] "GET /static/js/2.f20a604e.chunk.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2023 16:44:33] "GET /static/js/main.272b2057.chunk.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2023 16:44:33] "GET /datasets/datasets_config.json HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2023 16:44:33] "GET /manifest.json HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2023 16:44:33] "GET /datasets/testset/barData.json HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2023 16:44:33] "GET /datasets/testset/config.json HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2023 16:44:33] "GET /datasets/testset/metadata.json HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2023 16:44:33] "GET /datasets/testset/Colorfulness_Colors.json HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2023 16:44:33] "GET /datasets/testset/TSNE.json HTTP/1.1

files_report: datasets/datasets_config.json
files_report:files_report:files_report:  datasets/testset/config.json
files_report: datasets/testset/metadata.json manifest.json
datasets/testset/barData.json

files_report:files_report: datasets/testset/Colorfulness_Colors.json
files_report: datasets/testset/UMAP.json
files_report:  datasets/testset/PCA.json
datasets/testset/TSNE.json


127.0.0.1 - - [29/Mar/2023 16:44:33] "GET /datasets/testset/tile_0.png HTTP/1.1" 200 -


files_report: datasets/testset/tile_0.png


In [ ]:
#@title (optional) Run ngrok server
ngrok_Authtoken = 'YOUR_NGROK_TOKEN' #@param {type:"string"}
#@markdown >Note: **This will run until stopped!** Requieres an ngrok account. See: https://ngrok.com/

try:
  from pyngrok import ngrok
  from flask_ngrok import run_with_ngrok
  from flask import Flask, render_template,send_from_directory
except:
  print("Installing flask, flask_ngrok and pyngrok via Pip")
  %pip install flask flask_ngrok pyngrok
  from flask import Flask, render_template,send_from_directory
  from pyngrok import ngrok
  from flask_ngrok import run_with_ngrok
  
app = Flask(__name__,static_folder='/content/CSN/build/',template_folder='/content/CSN/build/')

ngrok.set_auth_token(ngrok_Authtoken)
run_with_ngrok(app)
@app.route('/<path:path>')
def send_report(path):
  # remove the replace in next to lines later later <-- important !!!!!!!!
  print("files",path)
  return send_from_directory('/content/CSN/build/', str(path))

@app.route('/CSN/static/<path:path>')
def send_report2(path):
  # remove the replace in next to lines later later <-- important !!!!!!!!
  print("files",path)
  return send_from_directory('/content/CSN/build/static/', str(path))
  
@app.route('/CSN/datasets/<path:path>')
def send_report3(path):
  # remove the replace in next to lines later later <-- important !!!!!!!!
  print("files",path)
  return send_from_directory('/content/CSN/build/datasets/', str(path))

@app.route("/")
def home():
    return render_template('index.html')
    
if __name__ == "__main__":
  app.run()

### (optional) Use as production web tool

We recommend to use GitHub for hosting your custom CSN version and an external server for hosting your image collection. Note that GitHub limits any dataset to 1000 files.

To deploy your version as a web tool in GitHub:

>Note: Make sure your GitHub branch is called `gh-pages` and has the GitHub Pages option set. See more about GitHub Pages here: https://pages.github.com/

1. clone the official CSN repository: https://github.com/Collection-Space-Navigator/CSN
2. install NVM: https://github.com/nvm-sh/nvm
3. install `node 14.21.2`: by running `nvm install v14.21.2`
4. replace the `CSN/build` folder with your own
5. in `package.json,` change `"homepage": ""` to your GitHub pages URL (e.g. `"homepage": "https://collection-space-navigator.github.io/CSN"`)
6. deploy the build folder to your GitHub pages by running `npm run deploy`

For more information and other deployment options, see https://create-react-app.dev/docs/deployment/

